# Collecting summoner IDs per Region, Division & Tier

In [ ]:
## Notes

! Display line numbers by clicking left of the green bar of the cell and pressing L. Has to be done on a per cell basis. 

Source: https://riot-api-libraries.readthedocs.io/en/latest/collectingdata.html

You can also use the League endpoints to get lists of ranked summoners. The positional league endpoints provide a paginated list of all summoners in a Tier + Division + Position (e.g. all ranked Diamond II Top laners). Alternatively, Canisback on the Discord currently hosts a list of league IDs that you can use to pull summoners from the leagues/{leagueId} endpoints. These lists are provided for free to the community for use, and may go down or stop being updated at any time.
    https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries

Helpful tutorial for this
- pulling specific data from API https://www.dataquest.io/blog/python-api-tutorial/ 

List of public API's for playing around another time
- https://github.com/public-apis/public-apis

Other (unused so far) tutorials:
- Writing & reading Json files https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/ 



In [ ]:
# MAIN CELL

# importing tools
import requests
import pandas as pd
import json
import numpy as np
import itertools

# create a formatted string of the Python JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# set desired variables
    # set region
    region = 'euw1'
    # set queue type
    queue_type = 'RANKED_SOLO_5x5'

tier_list = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
division_list = ['I', 'II', 'III', 'IV']

# iterate through the tiers & divisions - using itertools
c = list(itertools.product(tier_list, division_list))
for tierdivision_counter in c:
    print ('tierdivision_counter:', tierdivision_counter)

## creating the URL to pull the data from - WIP
# url_ids = ('https://' + region + '.api.riotgames.com/lol/league-exp/v4/entries/' + queue_type + '/' + tier_counter + '/' + str(division_counter) + '?page=' + page_counter + '&api_key=' + apikey)

url_ids = 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=5&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3'

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
    
# calling the API with the created URL
response_code = requests.get(url_ids,[])
response_json = response_code.json()
# jprint (response_json)

# check if response body is empty
if not bool(response_json):
    print("Awh, it's is empty.")
    ## stop the page counting there and go to the next division (continue?)
else:
    print("Yay! There is data.")

    #grab summonerId, summonerName, wins, & losses and appending them to a new list summoner_list
    summoner_list = []
    for summoner_info in response_json:
        summoner_list.append([summoner_info['summonerId'], summoner_info['summonerName'], summoner_info['wins'], summoner_info['losses']])
        #jprint(summoner_list)
        # creating a dataframe with the results
        summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses'])
 
 
# summoner_df.head(10)    #displaying the first 10 results of the dataframe
# summoner_df.tail(10)    #displaying the last 10 results of the dataframe

# summoner_dict = summoner_df.to_dict() # converting the dataframe into a dictionary
# summoner_dict.keys() # returns a list of all the available keys in the dictionary
# summoner_dict.values() # returns a list of all the available values in the dictionary



In [ ]:
# working for 1 url 

# importing tools
import requests
import pandas as pd
import json
import numpy as np
import itertools

# create a formatted string of the Python JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# set desired variables
    # set region
    region = 'euw1'
    # set queue type
    queue_type = 'RANKED_SOLO_5x5'

tier_list = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
division_list = ['I', 'II', 'III', 'IV']

# iterate through the tiers & divisions - using itertools
c = list(itertools.product(tier_list, division_list))
for tierdivision_counter in c:
    print ('tierdivision_counter:', tierdivision_counter)

## creating the URL to pull the data from - WIP
# url_ids = ('https://' + region + '.api.riotgames.com/lol/league-exp/v4/entries/' + queue_type + '/' + tier_counter + '/' + str(division_counter) + '?page=' + page_counter + '&api_key=' + apikey)

url_ids = 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=5&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3'

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
    
# calling the API with the created URL
response_code = requests.get(url_ids,[])
response_json = response_code.json()
# jprint (response_json)

# check if response body is empty
if not bool(response_json):
    print("Awh, it's is empty.")
    ## stop the page counting there and go to the next division (continue?)
else:
    print("Yay! There is data.")

    #grab summonerId, summonerName, wins, & losses and appending them to a new list summoner_list
        summoner_list = []
    for summoner_info in response_json:
        summoner_list.append([summoner_info['summonerId'], summoner_info['summonerName'], summoner_info['wins'], summoner_info['losses']])
        #jprint(summoner_list)
        # creating a dataframe with the results
        summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses'])
 
 
# summoner_df.head(10)    #displaying the first 10 results of the dataframe
# summoner_df.tail(10)    #displaying the last 10 results of the dataframe

# summoner_dict = summoner_df.to_dict() # converting the dataframe into a dictionary
# summoner_dict.keys() # returns a list of all the available keys in the dictionary
# summoner_dict.values() # returns a list of all the available values in the dictionary



In [ ]:



    * [ ] iterate through the pages
    * [ ] put everything in the right order (tier > division > page)
    * [ ] when done put the dataframe in a dict where the key is the tier+division


    # pseudo code
    for tier_list ...
        for division_list ... 
            for pages ... 
            if there is stuff in here, write it, 
            else if no data, stahp

In [ ]:
# wip to remove from as needed

# importing tools
import requests
import pandas as pd
import json
import numpy as np
import itertools

# create a formatted string of the Python JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# set desired variables
    # set region
    region = 'euw1'
    # set queue type
    queue_type = 'RANKED_SOLO_5x5'

tier_list = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
division_list = ['I', 'II', 'III', 'IV']

tier_list2 = ['GRANDMASTER', 'CHALLENGER']
division_list2 = ['I', 'II', 'III', 'IV']

# iterate through the tiers & divisions - using itertools
 c = list(itertools.product(tier_list2, division_list2))
 for tierdivision_counter in c:
     print ('tierdivision_counter:', tierdivision_counter)






for tierdivision_counter
 
    for pages








# generating url_ids
## for now its a manually populated list
url_ids = ['https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=3&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3', 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=4&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3', 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=5&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3']

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

    # calling the API with the created URL
    response_code = requests.get(url_ids,[])
    response_json = response_code.json()
    # jprint (response_json)



        #grab summonerId, summonerName, wins, & losses and appending them to a new list summoner_list
        summoner_list = []
        for summoner_info in response_json:
            summoner_list.append([summoner_info['summonerId'], summoner_info['summonerName'], summoner_info['wins'], summoner_info['losses']])
            #jprint(summoner_list)
            # creating a dataframe with the results
            summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses'])
    
    
    # summoner_df.head(10)    #displaying the first 10 results of the dataframe
    # summoner_df.tail(10)    #displaying the last 10 results of the dataframe

    # summoner_dict = summoner_df.to_dict() # converting the dataframe into a dictionary
    # summoner_dict.keys() # returns a list of all the available keys in the dictionary
    # summoner_dict.values() # returns a list of all the available values in the dictionary



In [ ]:
newlist = []
for x in range(10):
innerlist = []
for y in range(10):
innerlist.append(y)
newlist.append(innerlist)

jprint(newlist)

In [ ]:
url_ids = ['https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=3&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3', 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=4&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3', 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=5&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3']


for x in url_ids:
    print (x)

In [232]:
# building up again > trying to put everything together

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

###############


page_counter = 1
url_ids = []


# temp api key location
apikey = 'RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3'

# set variables set
region = 'euw1'
queue_type = 'RANKED_SOLO_5x5' 

# create variables that will be looped
tier_list = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
division_list = ['I', 'II', 'III', 'IV']

tier_list2 = ['GRANDMASTER', 'CHALLENGER'] # testing with these
division_list2 = ['I', 'II'] # testing with these

# iterate through the variables to create the dataset
for tier_counter in tier_list2:      # iterate through the tiers
    for division_counter in division_list2: # iterate through the divisions
        summoner_list = [] # create an empty summoner_list for each tier & division combination. data from more pages will go into the existing list
        print(tier_counter, division_counter)
        page_counter = 1
        url_id = ('https://' + region + '.api.riotgames.com/lol/league-exp/v4/entries/' + queue_type + '/' + str(tier_counter) + '/' + str(division_counter) + '?page=' + str(page_counter) + '&api_key=' + apikey) # create the URL  
        print (url_id)
        response_code = requests.get(url_id) # access the url
        print (response_code)
        ## I'm not getting into the if statement 
        if response_code == '[200]':
            print('if')
        else:  # else if there is no data, exit this loop
            print("else")
        
        
        
        



GRANDMASTER I
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3
<Response [200]>
else
GRANDMASTER II
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3
<Response [200]>
else
CHALLENGER I
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3
<Response [200]>
else
CHALLENGER II
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3
<Response [200]>
else


In [241]:
url_id = 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key=RGAPI-6afdc7c1-d52e-4d11-a5b6-992a29b533d3'
response_code = requests.get(url_id) # access the url

print (response_code.response)
## I'm not getting into the if statement 
if response_code == "<Response [200]>":
    print('if')
else:  # else if there is no data, exit this loop
    print("else")

AttributeError: 'Response' object has no attribute 'code'

In [ ]:
if response_code == 200:
            print('if')
        else:  # else if there is no data, exit this loop
            print("else")

In [ ]:
         and bool(response_json): # if the response is 200 and there is data, collect it. 
            response_json = response_code.json() # put the response in a json format
            print("Dictionary has data")
            #grab summonerId, summonerName, wins, & losses and appending them to a new list summoner_list
            for summoner_info in response_json:
                summoner_list.append([summoner_info['summonerId'], summoner_info['summonerName'], summoner_info['wins'], summoner_info['losses']])
                #jprint(summoner_list)

                ## get the data from this page
            else:  # else if there is no data, exit this loop
                print("Dictionary is empty")
                    
        else:   # if the response code is not 200, show the code and break the loop
            print(tier_counter, division_counter)
            print (response_code)
            break

    ## page 6 is not supposed to have data dammit 


In [ ]:
# creating a dataframe with the results
# summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses'])
## ? Do I need to pass the json response through a list before I can pass it into a dataframe? 
        